In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
model = tf.keras.models.load_model('action.h5')

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    b = 0
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
#     mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
#     mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections

    
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
actions = np.array(['normal', 'drink', 'phone'])

def prob_viz(res, actions, input_frame, colors, recheck, action_s):
    output_frame = input_frame.copy()
#     print(res)
    if recheck == True:
#         print(action_s)
        if action_s == 'normal':
            res[0] = 0.95
            res[1] = 0.025
            res[2] = 0.025
        elif action_s == 'drink':
            res[0] = 0.95
            res[1] = 0.025
            res[2] = 0.025
        elif action_s == 'phone':
            res[0] = 0.95
            res[1] = 0.025
            res[2] = 0.025
        for num, prob in enumerate(res):
            cv2.rectangle(output_frame, (0,80+num*40), (int(prob*100), 110+num*40), colors[num], -1)
            cv2.putText(output_frame, actions[num], (0, 105+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [24]:
# 1. New detection variables
sequence = []
sentence = []
clss = []
threshold = 0.8
actions = np.array(['normal', 'drink', 'phone'])

cap = cv2.VideoCapture('outfromlocation.avi')


# Get the frames per second
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)

# Get the total numer of frames in the video. value same as image frame
frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print(frame_count)

#Time of video(sec)
duration = frame_count / fps
print(duration)


# bgsubknn = cv2.createBackgroundSubtractorKNN(40) # มักจะดีกว่า , default จะเก็บ buffer 400 - 500 grame
# L1 = np.r_[50:53]
# L2 = np.r_[60:63]

L1_status, L2_status = False, False
count = 0
relay = [0, 10, 0, 0, 0] #p[count, star, fg1, fg2, out-area]
relay_status = False
out_area = False

timer = 0
process_time = 0
cycle_time = []
std_time = 9
green = (0, 255, 0)
red = (0, 0, 255)


bgsubknn1 = cv2.createBackgroundSubtractorKNN(40)
bgsubknn2 = cv2.createBackgroundSubtractorKNN(40)
y1 = 140
y2 = 200
x1 = 50
x2 = 200

yy1 = 170
yy2 = 250
xx1 = 430
xx2 = 500

process_n = 0 # flase=process, True=double-process
process_s = 0

c=0
u=1

start_point, end_point = (300,25),(450,325)

# setting write
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('out.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (frame_width,frame_height))


25.0
0.0
0.0


In [ ]:
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        if not ret:
            break
        
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
#         draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        # Start===========================================================================
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
#             print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            
            if len(clss) <= 90:
                clss.append(actions[np.argmax(res)])
                recheck = False
                action_s = actions
#                 actions = 'normal'
            else:
                clss.append(actions[np.argmax(res)])
                if len(clss) > 90:
                    clss = clss[1:]
                action_s = max(clss,key=clss.count)
                recheck = True
             
            image = prob_viz(res, actions, image, colors, recheck, action_s)
            
#         cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
#         cv2.putText(image, ' '.join(sentence), (3,30), 
#                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        try:
            x_center = round(((results.pose_landmarks.landmark[11].x*frame_width) + (results.pose_landmarks.landmark[12].x*frame_width))/2)
            y_center = round(((results.pose_landmarks.landmark[11].y*frame_height) + (results.pose_landmarks.landmark[12].y*frame_height))/2)
#             cv2.circle(image, (x_center,y_center), radius=5, color=(0, 255, 0), thickness=-1)
# #           print(x_center, y_center)
            if x_center > 450 or x_center < 300:
                out_area = True
                relay[4] += 1
            else:
                out_area = False
                relay[4] = 0
        except:
#             print('=')
            pass
        

        # Start===========================================================================
        frame = cv2.GaussianBlur(frame ,(5,5),0)
        
        c += 1
        timer += 1
        crop_frame1 = frame[y1:y2, x1:x2]
        crop_frame2 = frame[yy1:yy2, xx1:xx2]

        fg1 = bgsubknn1.apply(crop_frame1)
        fg2 = bgsubknn2.apply(crop_frame2)

        relay[2] += 1 # action
        relay[3] += 1
#         print(relay[2])
        if np.sum(fg1) > 400000 and c > 20 and relay[2] > 100:
            L1_status = True
            process_n += 1

            relay[2] = 0
        else:
            u=1


        if np.sum(fg2) > 100000 and c > 70 and relay_status == True and relay[3]>100:
            L2_status = True
            relay[3] = 0
            
            
        else:
            if np.sum(fg2) > 400000:
                L2_status = False
                relay[3] = 0
            u=1
        
        print(L1_status, L2_status, relay_status, relay[2], np.sum(fg1), np.sum(fg2))
        if L1_status and L2_status:
            L1_status = False
            L2_status = False
            if relay_status:
                relay_status = False
                L1_status = False
                L2_status = False
                process_n = 0
                count += 1
                print(count)
                process_time = timer/fps
                cycle_time.append(process_time)
                print(cycle_time)
                print(sum(cycle_time)/len(cycle_time))
                timer = 0
        process_s = process_n
        
        cv2.putText(image, 'Timer: ' + str(round(timer/fps)) + ' Sec', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        cv2.putText(image, 'Std. time = ' + str(round(std_time)) + ' Sec', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        if process_time > std_time:
            color = red
            cv2.putText(image, 'process_time = ' + str(round(process_time)) + ' Sec', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5,color, 2)
        else:
            color = green
            cv2.putText(image, 'process_time = ' + str(round(process_time)) + ' Sec', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        if process_s == 0:
            txt = 'none'
        elif process_s == 1:
            txt = 'process'
        elif process_s >= 2:
            txt = 'double-process'

        cv2.putText(image, 'status: ' + txt, (480, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
        if out_area == True and (relay[4]/fps) > 2:
            cv2.rectangle(image, start_point, end_point, red, 2)
        else: 
            cv2.rectangle(image, start_point, end_point, green, 2)
        
        # condition relay
        if relay_status == False:
            relay[0] += 1


        if relay[0] >= relay[1]:
            relay_status = True
            relay[0] = 0

        out.write(image)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

False False False 1 2295000 1428000
False False False 2 2295000 1428000
False False False 3 2184330 1428000
False False False 4 2295000 1428000
False False False 5 2295000 1428000
False False False 6 2295000 1428000
False False False 7 2295000 1428000
False False False 8 287857 0
False False False 9 254033 0
False False False 10 249756 0
False False True 11 313020 0
False False True 12 339908 0
False False True 13 330345 0
False False True 14 327436 0
False False True 15 310121 255
False False True 16 305054 1275
False False True 17 293474 0
False False True 18 269650 0
False False True 19 241494 0
False False True 20 223501 0
False False True 21 207438 0
False False True 22 226158 0
False False True 23 238527 1911
False False True 24 273070 3184
False False True 25 281724 2929
False False True 26 295617 4833
False False True 27 296488 4322
False False True 28 272002 2794
False False True 29 255766 6733
False False True 30 264667 8384
False False True 31 264401 10922
False False True 3

False False True 95 68751 0
False False True 96 60979 0
False False True 97 47483 2040
False False True 98 35508 0
False False True 99 31819 255
False False True 100 27236 0
False False True 101 22271 127
False False True 102 18326 127
False False True 103 18838 0
False False True 104 15786 0
False False True 105 12220 127
False False True 106 10189 254
False False True 107 7134 508
False False True 108 6114 254
False False True 109 6238 635
